# AutoEncoders

Dataset: MovieLens movie ratings dataset.

Generated a movie Recommender System which predicts the rating (from 1 to 5) a user gives to a movie.

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


In [5]:
# The dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')


In [6]:
# training set and test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [7]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [8]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [9]:
# Architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [10]:
# Training the Stacked AutoEncoder
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 1.77217701284
epoch: 2 loss: 1.0965782176
epoch: 3 loss: 1.05347074123
epoch: 4 loss: 1.03830566072
epoch: 5 loss: 1.0308382606
epoch: 6 loss: 1.02667183158
epoch: 7 loss: 1.0237787274
epoch: 8 loss: 1.02207068095
epoch: 9 loss: 1.02044118022
epoch: 10 loss: 1.01979012286
epoch: 11 loss: 1.01896442546
epoch: 12 loss: 1.01851487619
epoch: 13 loss: 1.01801173533
epoch: 14 loss: 1.01762656948
epoch: 15 loss: 1.01728925065
epoch: 16 loss: 1.01685586959
epoch: 17 loss: 1.01674925257
epoch: 18 loss: 1.01640375847
epoch: 19 loss: 1.01637733341
epoch: 20 loss: 1.01596886418
epoch: 21 loss: 1.01612099602
epoch: 22 loss: 1.01597204281
epoch: 23 loss: 1.01591820193
epoch: 24 loss: 1.01581167777
epoch: 25 loss: 1.01567057062
epoch: 26 loss: 1.0155827336
epoch: 27 loss: 1.01549701092
epoch: 28 loss: 1.01490225849
epoch: 29 loss: 1.01272417045
epoch: 30 loss: 1.01179294402
epoch: 31 loss: 1.00931321161
epoch: 32 loss: 1.00921891085
epoch: 33 loss: 1.00506888943
epoch: 34 loss: 1.00580

In [11]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.969294267437
